# The SignalSystem and recorded Sessions in depth

### Update rate and data feeds

So what happens when we create a system and run it?

In [ ]:
from genki_signals.sources import Sampler, MouseSource
from genki_signals.system import System

source = Sampler({'mouse_position': MouseSource()}, sample_rate=50)
system = System(source, [])

system.start()

Behind the scenes, the `system` object has spawned a thread that runs in a loop until we call `system.stop()`. The loop simply queries the source for new data points, and computes all of the specified signal functions. If we look at the signature for the `System` constructor, we see that there is one more optional parameter, called `update_rate` which defaults to 25.

In [ ]:
System?

The `update_rate` defines how frequently (in Hz) the system checks for new data from the source. The default value 25 is selected to make vlisualizations run smoothly enough, without sacrificing too much performance. In our example, the `source` has a sample rate of 50, so we expect the system loop to receive two data points on average. Since all signal functions are implemented in `numpy`, doing computations less often, on larger batches of data, is faster. If you need higher throughput, and don't care as much about latency in visualization, consider lowering the `update_rate`.

The way the system communicates the computed data to the outside world is through _data feeds_. A data feed is simply a callback with some ID. We can _register_ a data feed using `register_data_feed`:

In [ ]:
class Counter:
    def __init__(self):
        self.count = 0
        
    def __call__(self, data):
        self.count += 1
        
counter = Counter()
system.register_data_feed('counter', counter)

In this case, `'counter'` is the ID - it can be a string, or some other type (just needs to be hashable). 

In [ ]:
counter.count

Front ends are implemented using data feeds, and there is no limit to how many feeds a system can have. It is possible, for example, to have one front end running a web server for a dashboard site, and another one displaying visualizations in a jupyter notebook, both showing data from the same underlying system.

Everything in Genki Signals is designed to be dynamic. For example, we can add data feeds and signal functions to a running system:

In [ ]:
from genki_signals import functions as f

system.add_derived_signal(f.Logarithm('mouse_position', name='log_mouse_position'))
system.functions

### Recorders

The default behaviour is to record data in `.pickle` files. This is convenient when every part of your pipeline is written in Python and can depend on Genki Signals, it allows us to write `DataBuffer` objects directly to binary files. 

Sometimes, however, it is useful to be able to record data in a different format. For that, the `start_recording` function takes in an optional parameter, `recorder`:

In [ ]:
from genki_signals.recorders import CsvFileRecorder

base_path = "mouse_data"
recorder = CsvFileRecorder(base_path + "/raw_data.csv")
system.start_recording(base_path, recorder=recorder)

In [ ]:
system.stop()

In [ ]:
!head mouse_data/raw_data.csv

Genki Signals provides recorders to write `.csv`, `.parquet`, and `.wav` files. If you need something else, writing your own recorder is simply a matter of inheriting from `Recorder` and implementing `write()` and `stop()`:

In [ ]:
from genki_signals.recorders import Recorder

class NoDataJustNamesRecorder(Recorder):
    def __init__(self, path):
        self.path = path
        self.f = open(self.path, 'wa')
    
    def write(self, data):
        self.f.writelines(list(data.keys()))
    
    def stop(self):
        self.f.close()

### Sessions, signal function parameters

In the recorded data we read from the csv file above, note that our signal function `log_mouse_position` is **not** written to the file. The reason for this is that all signal functions are assumed to be deterministic, which means we can recompute them at any time, and storing the computed values is redundant.  Signal functions are also serializable and written with the session, so we can derive the exact data that was being computed at the time of recording:

In [ ]:
from genki_signals.session import Session

sesh = Session.from_filename("mouse_data")
sesh.functions

This highlights an important point: since we only record the raw, _source of truth_-data, the parameters of the preprocessing pipeline (just the base of the logarithm in this case) can be changed later, and when we are doing ML with the data, they can be _treated as hyperparameters_. 

Changing the base of a logarithm is perhaps not a very useful hyperparameter, but suppose this was some low-pass filter, with parameters such as cutoff frequency and order. Instead of manually tuning the filter to something that looks reasonable in visualization (although this is of course still possible) - one can use the filter parameters as hyperparameters and use the ones that work best in an end to end ML pipeline.

The signal functions used is not the only metadata stored about a session:

In [ ]:
sesh.metadata

These are just the defaults. When calling `start_recording`, you can add arbitrary key-value pairs that will be stored as metadata. When you have collected multiple data sessions, these can be important. For example:

- When recording data from multiple users you might want to make sure that no data session from a user in the training set appears in the test set. For this you need to add a metadata field about which user is being recorded in each session.

- You might want to include information on factors such as gender, race, etc. This can help you analyze and eradicate undesired bias in the final model.

To get the session data, including the computed signal functios, call `get_data()`:

In [ ]:
sesh.get_data()

One more thing to note here, in our case `'mouse_position'` is a 2D tensor. In file formats like csv, each column can only be a 1d series, so in the written file the data is split into `mouse_position_0` and `mouse_position_1` and then parsed into the 2D signal again when the file is read. Genki Signals does this automatically and it works for up to 4D tensor signals (with one dimension being reserved for time, so e.g. video with shape `(width, height, channels, time)` works). This does mean however that one needs to be careful with including numbers in signal names, if you need a tabular file format it is probably best to not include numbers in any signal names.